<a href="https://colab.research.google.com/github/prakharsingh97/Operations_Department/blob/main/Operations_Department.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#What are we going to learn 


1.   CNN 
2.   Deep Learning 
            1.  Pretrained Weights 
            2.  From Scratch 
3.   Resnet 
4.   Transfer Learning  -> Reduces Training Time
5.   Evalutaion 



# Case Study


#Importing Dataset


In [2]:
import os
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, optimizers
#keras api calls tf and use to implement deep learning
from tensorflow.keras.applications.resnet50 import ResNet50
# Residual Network SOA pre- trained network
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#generate batches of images
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
#EarlyStopping prvents overfitting
#ModelCheckpoint select best model 
import matplotlib.pyplot as plt
import seaborn as sns
#data visualization
import pandas as pd


#Explore Dataset

In [3]:
#importing directory 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_ray_directory = 